In [1]:
from langchain_community.document_loaders import PyPDFLoader

loaderI = PyPDFLoader("files/goog-10-k-2023 (1).pdf")
TDI = loaderI.load()

loaderII = PyPDFLoader("files/tsla-20231231-gen.pdf")
TDII = loaderII.load()

loaderIII = PyPDFLoader("files/uber-10-k-2023.pdf")
TDIII = loaderIII.load()



In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
documents = text_splitter.split_documents(TDI)



In [3]:
# embeddings

from langchain_community.embeddings import CohereEmbeddings
from langchain_community.vectorstores import Chroma

db = Chroma.from_documents(documents[:20],CohereEmbeddings(cohere_api_key='COHERE_API_KEY')) 

d:\Internship\pdf_summarizabe_chatbot\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `CohereEmbeddings` was deprecated in LangChain 0.0.30 and will be removed in 0.3.0. An updated version of the class exists in the langchain-cohere package and should be used instead. To use it run `pip install -U langchain-cohere` and import as `from langchain_cohere import CohereEmbeddings`.
  warn_deprecated(
Retrying langchain_community.embeddings.cohere.CohereEmbeddings.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised UnauthorizedError: status_code: 401, body: {'message': 'invalid api token'}.
Retrying langchain_community.embeddings.cohere.CohereEmbeddings.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised UnauthorizedError: status_code: 401, body: {'message': 'invalid api token'}.


UnauthorizedError: status_code: 401, body: {'message': 'invalid api token'}

In [6]:
query="by the name of the book"
result = db.similarity_search(query)
result[0].page_content

'traditional retail seasonality, which are likely to cause fluctuations in our quarterly results;\n•the sufficiency of our sources of funding;\n•our potential exposure in connection with new and pending investigations, proceedings, and other \ncontingencies, including the possibility that certain legal proceedings to which we are a party could harm our \nbusiness , financial condition, and operating results;\n•our expectation that we will continue to face heightened regulatory scrutiny and cha nges in regulatory \nconditions, laws, and public policies,  which could affect our business practices and financial results;Table of Contents Alphabet Inc.\n3.'

In [7]:
from langchain_community.llms import Ollama

llm= Ollama(model="llama3")
llm

Ollama(model='llama3')

In [8]:
from langchain_core.prompts import ChatPromptTemplate

prompt= ChatPromptTemplate.from_template("""
Answer the following question based  only on the provided context.
Think step by step before providing  a detailed answer.
I will tip you $1000 if user finds the answer helpful.
<context>
{context}
</context>
Question: {input}""")


In [9]:
# Creatnig a chain

from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain= create_stuff_documents_chain(llm,prompt)


In [10]:
#retriver

retriver= db.as_retriever()
retriver


VectorStoreRetriever(tags=['Chroma', 'CohereEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x000002DC7FE4ADE0>)

In [11]:
#creating retriver chain

from langchain.chains import create_retrieval_chain

retreval_chain=create_retrieval_chain(retriver,document_chain)


In [14]:
response = retreval_chain.invoke({"input":"Arrangements with Multiple Performance Obligations"})


In [15]:
response['answer']

'A $1000 tip waiting for me!\n\nBased on the provided context, I\'ll answer your question step by step.\n\nThe relevant section is Item 7, Management\'s Discussion and Analysis of Financial Condition and Results of Operations, which discusses "the amount and timing of revenue recognition from customer contracts with commitments for performance obligations..."\n\nThis implies that Alphabet Inc. has arrangements with multiple performance obligations, as they mention recognizing revenue from "customer contracts" with such commitments. The use of the phrase "performance obligations" suggests that these contracts involve complex transactions with multiple components or milestones that need to be fulfilled before revenue is recognized.\n\nTherefore, my answer is:\n\nYes'